![alt_text](http://www.gtec.at/var/plain_site/storage/images/media/images/g.nautilus-g.kids-400x240/351775-1-eng-GB/g.Nautilus-g.Kids-400x240.jpg 'Child having an EEG test done')

# Welcome to the EEG Tutorial!
#### By the end of this tutorial, you will be able to...

_1. Understand what an EEG is._

_2. Recognize the five different brain waves and when they occur._

_3. Examine your own brain data!_

---

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to Hide/Show the code."></form>''')

In [ ]:
from IPython.display import Javascript
Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+6)')

## So, what is an EEG and why should we be learning about it?

EEG stands for __electroencephalogram__, which is a test that records the electrical signals produced by the brain. Our brain cells communicate using electrical signals, and we can measure some of these signals using special electrodes (like the ones the children are wearing in the image above). A signal may be classified by its __frequency__ (number of peaks measured per second) and __amplitude__ (the strength of each peak).

![alt_text](https://i1.wp.com/www.mindovermenieres.com/wp-content/uploads/2015/12/brainwave-chart-01.png?w=600 "Types of EEG Waves")

---

## That's cool, but what do these waves mean? 

Each of the five waves listed above represent a different brain __state__. For example, the __delta__ waves are observed when a person is in __deep sleep__. The short video below will explain each wave and the brain state associated with it:

In [ ]:
from IPython.display import YouTubeVideo
# a short video about brain waves
# Video credit: Brainfacts.org.
YouTubeVideo('8CejGESrRkc')

---

## Why are EEG tests important? What's the point?

An EEG test is capable of detecting brain disorders such as __seizures__, __dizziness__, __epilepsy__, and even __brain tumors__. Many disorders are detected by noting an abnormal spike/pattern in the brain waves as they're being measured. The following image shows normal brain wave activity followed by brain wave activity during a seizure:

![alt_text](http://rrapid.leeds.ac.uk/ebook/assets/images/illustrations/EEG.png 'Seizure EEG')

---

## Awesome, when can I look at my brain waves?

Right now! We probably won't see any __delta__ or __theta__ waves (unless you were very committed and _fell asleep_ while taking measurements), but if you closed your eyes and relaxed at any point, you may catch an __alpha__ wave somewhere in your data! Or, if you were _really_ deep in thought or focused on a problem, you could even find a __gamma__ wave!  

Below you will see a button that will ask you to select the data file of your EEG recording.  

After you select your file, four plots (one for each of the electrodes you placed on your head) will appear below. Each plot has __four__ sliders:  
&emsp;&emsp;(1) x position; (2) width; (3) y position; (4) height of your view window
    
Each plot will automatically update as you move any of the sliders.  

If you're feeling extra adventurous, click the button at the top of this notebook that says 'Click here to toggle the code.' to show all the code used to create this notebook. You can modify this code if you wish to add any features to the plots, change the colors, etc.  

If you wish to __restart__ the analysis for the same recording, click the 'Restart' button.  

If you wish to __start over__ with a new EEG file, simply click the 'Select Data File' button.  

#### Good luck and enjoy the _waves_!

---

In [ ]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive_output,HBox,VBox,Label,Layout
from IPython.display import Javascript,display
from tkinter import Tk, filedialog
import os
import warnings
warnings.filterwarnings('ignore')

root = Tk()
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

file_name = ''
def select_files(*arg):
    global file_name
    filename = filedialog.askopenfilename(initialdir=os.getcwd(),title="Select data file",filetypes=[("Text files","*.txt")])
    if filename:
        file_name = filename
        print('File selected: '+file_name);
        display(Javascript('IPython.notebook.execute_cells([IPython.notebook.get_selected_index()+1])'))

SelectFileButton = widgets.Button(description='Select Data File',button_style='primary')
SelectFileButton.on_click(select_files)
display(SelectFileButton)

In [ ]:
def restart(*arg):
    if not file_name:
        print('File not selected')
    elif os.path.exists(file_name):
        print('Restarted')
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+5)'))
    else:
        print('File does not exist.')

restart_button = widgets.Button(description='Restart',button_style='primary')
restart_button.on_click(restart)
display(restart_button)

Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+5)')

In [ ]:
# ...And reading in our data:
# We want to read in number of channels and sampling rate from the first few rows, then drop them
# and we want to give our data some column labels:
headers = ['Index','EEG_1','EEG_2','EEG_3','EEG_4','ACC_1','ACC_2','ACC_3','Timestamp_1','Timestamp_2']
eeg_data = pd.read_csv(file_name,names=headers)
NChannels = [int(s) for s in eeg_data.iloc[1,0].split() if s.isdigit()][0] # number of channels
Fs = [float(s) for s in eeg_data.iloc[2,0].split() if s.replace('.','',1).isdigit()][0] # sampling rate
eeg_data.dropna(inplace=True) # drop first few lines possible NaN values
eeg_data.reset_index(inplace=True,drop=True)
eeg_col = ['EEG_'+str(i) for i in range(1,NChannels+1)] # column names

print('File loaded successfully.')
# Uncomment the line below to print a prevhiew of the data
# print(eeg_data)

eeg = eeg_data[eeg_col]
N = eeg.shape[0] # Total number of time samples
T = eeg.index.values/Fs # Time samples (second)
channel_site = ['Left Frontal Lobe','Right Frontal Lobe','Left Occipital Lobe','Right Occipital Lobe']
channel = ['1','2','3','4']

---

## Raw EEG signal of muti-channel recordings
Select channels and view the raw EEG. Use the sliders to control axes. To update faster, disable 'continuous update'.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive_output,HBox,VBox,Label,Layout
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline

tpseg = 4 # length of segment (second) for spectral analysis
npseg = 2**np.ceil(np.log2(tpseg*Fs)) # samples per segment
minseg = npseg/Fs # minimun length of segment required

[xmin,xmax,ymin,ymax] = [T[0],T[-1],eeg.min().min(),eeg.max().max()]
[W,H] = [xmax-xmin,ymax-ymin]
def plot_eeg(channel,px,py,pw,ph):
    nchannel = len(channel)
    if nchannel == 0:
        print('Channels not selected.')
        return()
    
    xx = np.array([px-1/pw/2,px+1/pw/2])
    yy = np.array([py-1/ph/2,py+1/ph/2])
    [x1,x2] = [max(np.floor(xx[0]*N).astype(int),0) , min(np.ceil(xx[1]*N).astype(int),N)+1]
    
    plt.figure(figsize=(12,5*nchannel))
    for i in range(nchannel):
        plt.subplot(nchannel,1,i+1)
        plt.plot(T[x1:x2],eeg['EEG_'+channel[i]][x1:x2],'b')
#         plt.plot(T,eeg['EEG_'+channel[i]],'b')
        plt.xlim(xmin+W*xx)
        plt.ylim(ymin+H*yy)
        plt.grid()
        if i==nchannel-1: plt.xlabel('Time (sec)')
        plt.ylabel('Voltage $(\mu V)$')
        plt.title(channel_site[int(channel[i])-1]+' (EEG_'+channel[i]+')')
    plt.show()

xslider = Layout(width='500px')
yslider = Layout(height='250px')
w_chan = widgets.SelectMultiple(options=list(zip(channel_site,channel)),rows=NChannels,value=channel)
w_px = widgets.FloatSlider(value=0.5,min=0.5,max=0.5,step=minseg/T[-1]/10,readout=False,orientation='horizontal',layout=xslider)
w_py = widgets.FloatSlider(value=0.5,min=0.5,max=0.5,step=0.001,readout=False,orientation='vertical',layout=yslider)
w_pw = widgets.FloatLogSlider(value=1,base=10,min=0,max=np.log10(T[-1]/minseg),step=0.02,readout=False,orientation='horizontal',layout=xslider)
w_ph = widgets.FloatLogSlider(value=1,base=10,min=0,max=2,step=0.02,readout=False,orientation='vertical',layout=yslider)
w_update = widgets.ToggleButton(value=True,description='Continuous update',button_style='info',icon='times')

def cont_update(*args):
    if w_update.value:
        w_update.icon = 'times'
        w_px.continuous_update = True; w_py.continuous_update = True
        w_pw.continuous_update = True; w_ph.continuous_update = True
    else:
        w_update.icon = 'repeat'
        w_px.continuous_update = False; w_py.continuous_update = False
        w_pw.continuous_update = False; w_ph.continuous_update = False
w_update.observe(cont_update,'value')

def update_px(*arg):
    w_px.min = max(1/w_pw.value/2,w_px.value-1/w_pw.value)
    w_px.max = min(1-1/w_pw.value/2,w_px.value+1/w_pw.value)
w_pw.observe(update_px,'value')
w_px.observe(update_px,'value')

def update_py(*arg):
    w_py.min = max(1/w_ph.value/2,w_py.value-1/w_ph.value)
    w_py.max = min(1-1/w_ph.value/2,w_py.value+1/w_ph.value)
w_ph.observe(update_py,'value')
w_py.observe(update_py,'value')

around = Layout(align_items='flex-start',justify_content='space-around')
out = interactive_output(plot_eeg,{'channel':w_chan,'px':w_px,'py':w_py,'pw':w_pw,'ph':w_ph})
ui = VBox([ Label('Select Channels (select multiple by holding ctrl/shift + click)'),w_chan,
           HBox([VBox([Label('Zoom in/out'),Label('X positon')]),VBox([w_pw,w_px]),w_update],layout=around),
           HBox([out, VBox([HBox([Label('Y positon'),Label('Zoom in/out')]),HBox([w_py,w_ph])]) ],layout=around) ])

display(ui)

In [ ]:
def Continue(*arg):
    display(Javascript('IPython.notebook.execute_cells_below()'))

continue_button = widgets.Button(description="Continue",button_style='primary')
continue_button.on_click(Continue)
display(continue_button)

In [ ]:
print('If you want to cut out a segment from the EEG recording, please enter your desired interval in second (e.g. "5 12.5") and press [ENTER].\nOtherwise, press enter to skip: ')
while True:
    interval = input()
    interval = [float(s) for s in interval.replace(',',' ').split() if s.replace('.','',1).isdigit()]
    if len(interval)<2:
        print('No interval specified. Entire recording will be used.');
        interval = T[[0,-1]]
    else:
        interval = np.array([max(interval[0],0),min(interval[1],T[-1])])
    if interval[1]-interval[0]<=minseg:
        print('Selected segment %.2f-%.2f second is too short for further analysis.Please choose a segment longer than %.2f seconds.'
              %(interval[0],interval[1],minseg))
    else:
        print('%.2f-%.2f second will be cut out for further analysis.' % tuple(interval)); break

idx = np.nonzero(np.logical_and(T>=interval[0], T<=interval[1]))
eeg = eeg.iloc[idx]
N = eeg.shape[0]
T = T[idx]
xmin = T[0]
W = T[-1]-xmin

---

## Power spectral density of EEG recordings
Calculate the power spectral density of EEG over the whole recording using Welch's method. Divide the power spectrum into 6 bands of different rhythms.

In [ ]:
from scipy import signal

waves = ['Delta','Theta','Alpha','Beta','Gamma','High Gamma']
freq_band = np.array([[1.,4.],[4.,8.],[8.,12.],[12.,30.],[30.,70.],[70.,100.]])
nband = len(waves)
fshow = 100 # Maximun frequency to display (Hz)
eeg_np = eeg.values # convert to numpy array
eeg_np -= eeg_np.mean(axis=0) # remove DC component
jet = plt.get_cmap('jet') # colormap
clr = jet(np.linspace(0,1,nband)) # colors

Ff, Pxx = signal.welch(eeg_np,Fs,nperseg=npseg,axis=0) # welch's power spectral density estimation
Pxx[[0,-1],:]*=2
plt.figure(figsize=(8,6))
plt.semilogy(Ff,Pxx)
plt.xlim(0,fshow)
yl = plt.gca().get_ylim()
for i in range(nband):
    ytxt = yl[0]*(yl[1]/yl[0])**(np.array([0.9,0.88])-0.05*i)
    plt.semilogy(freq_band[i,0]*np.ones(2),yl,color=clr[i])
    plt.semilogy(freq_band[i,1]*np.ones(2),yl,color=clr[i])
    plt.semilogy(freq_band[i,:],ytxt[1]*np.ones(2),color=clr[i])
    plt.text(freq_band[i,0],ytxt[0],waves[i],color=clr[i])
    print(waves[i]+': %.0f-%.0fHz ' %(freq_band[i][0],freq_band[i][1]),end = '   ')
plt.ylim(yl)
plt.legend(eeg_col,loc=1)
plt.title('Power spectral density')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD $(\mu V^2/Hz)$')
plt.show()

## Band-pass filter the EEG to get waves
Design Butterworth filter for each rhythm and apply Hilbert transform.

In [ ]:
eeg_filt = eeg_HBamp = np.empty( shape=(N,NChannels,nband) )
eeg_power = np.empty( shape=(N,NChannels,nband) )
for i in range(nband):
    if freq_band[i,1]>=Fs/2:
        bFilt,aFilt = signal.butter(2,freq_band[i,0]*2/Fs,btype='highpass') # High-pass filter if cutoff frequency exceed Nyquist frequency
    else:
        bFilt,aFilt = signal.butter(2,freq_band[i]*2/Fs,btype='bandpass') # Design filters
    eeg_filt[:,:,i] = signal.filtfilt(bFilt,aFilt,eeg_np,axis=0) # Filtered EEG
    for j in range(NChannels):
#         eeg_power[:,j,i] = np.sqrt(signal.convolve(eeg_filt[:,j,i]**2,np.ones(int(npseg)),mode='same')/npseg)
        eeg_power[:,j,i] = np.sqrt(signal.convolve(eeg_filt[:,j,i]**2,np.ones(int(npseg)),mode='full')/npseg)[:N] # Accumulated Power
eeg_HBamp = np.absolute(signal.hilbert(eeg_filt,axis=0)) # Hilbert tranform

def pow2db(x):
    return 10*np.log10(x)
FF,t,Sxx = signal.spectrogram(eeg_np[:,0],Fs,nperseg=int(npseg/4),noverlap=int(npseg/16)) # Spectrogram
fidx = np.nonzero(FF<=fshow)
F = FF[fidx]
SxxdB = pow2db(Sxx[fidx])

Select a channel and a rhythm to display its raw EEG, filtered EEG, accumulated power, power spectral density and spectrogram.

In [ ]:
Ymin = [eeg_np.min(axis=0),-eeg_HBamp.max(axis=0)]
HH = [eeg_np.max(axis=0)-Ymin[0],-2*Ymin[1],eeg_power.max(axis=0)]
def plot_waves(channel,band,px,py1,py2,pw,ph1,ph2):
    xx = np.array([px-1/pw/2,px+1/pw/2])
    y1 = np.array([py1-1/ph1/2,py1+1/ph1/2])
    y2 = np.array([py2-1/ph2/2,py2+1/ph2/2])
    [x1,x2] = [max(np.floor(xx[0]*N).astype(int),0) , min(np.ceil(xx[1]*N).astype(int),N)]
    
    plt.figure(figsize=(12,20))
    # Raw signal
    plt.subplot(4,1,1)
    plt.title(waves[band]+' wave of '+channel_site[channel]+' (EEG_'+str(channel+1)+')')
    plt.plot(T[x1:x2],eeg_np[x1:x2,channel],'k')
    plt.xlim(xmin+W*xx)
    plt.ylim(Ymin[0][channel]+HH[0][channel]*y1)
    plt.grid()
    plt.xlabel('Time (sec)')
    plt.ylabel('Voltage $(\mu V)$')
    plt.legend(['Raw EEG'],loc=1)
    
    # Filtered signal
    plt.subplot(4,1,2)
    plt.plot(T[x1:x2],eeg_filt[x1:x2,channel,band],'b')
    plt.plot(T[x1:x2],eeg_HBamp[x1:x2,channel,band],'m')
    plt.xlim(xmin+W*xx)
    plt.ylim(Ymin[1][channel,band]+HH[1][channel,band]*y2)
    plt.grid()
    plt.xlabel('Time (sec)')
    plt.ylabel('Voltage $(\mu V)$')
    plt.legend(['Band-pass filtered EEG','Hilbert transform amplitude'],loc=1)
    # Accumulated average power in band
    plt.subplot(4,1,3)
    plt.plot(T[x1:x2],eeg_power[x1:x2,channel,band],color=clr[band])
    plt.xlim(xmin+W*xx)
    plt.ylim(0,HH[2][channel,band])
    plt.grid()
    plt.xlabel('Time (sec)')
    plt.ylabel('Voltage $(\mu V)$')
    plt.legend([waves[band]+' power'],loc=1)
    # Power spectral density
    f,pxx = signal.welch(eeg_np[x1:x2,channel],Fs,nperseg=npseg,axis=0) # PSD with the display window
    plt.subplot(4,1,4)
    plt.plot(Ff,pow2db(Pxx[:,channel]),'g')
    plt.plot(f,pow2db(pxx),color=clr[band])
    plt.xlim(0,fshow)
    yl = plt.gca().get_ylim()
    plt.plot(freq_band[band,0]*np.ones(2),yl,color=clr[band,:3]*.7)
    plt.plot(freq_band[band,1]*np.ones(2),yl,color=clr[band,:3]*.7)
    plt.text(freq_band[band,0],yl[0]+(yl[1]-yl[0])*(0.9-0.05*band),waves[band],color=clr[band,:3]*.3)
    plt.ylim(yl)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD (dB/Hz)')
    plt.legend(['total PSD','PSD of displayed duration'],loc=1)
    plt.show()
    # Spectrogram
    plt.figure(figsize=(14,5))
    plt.pcolormesh(xmin+t,F,SxxdB,cmap='winter')
    plt.colorbar().set_label('Power/Frequency (dB/Hz)')
    plt.yscale('log')
    plt.plot(xmin+W*xx,freq_band[band,0]*np.ones(2),color=clr[band])
    plt.plot(xmin+W*xx,freq_band[band,1]*np.ones(2),color=clr[band])
    plt.text(xmin+W*xx[0],freq_band[band,0]*1.05,waves[band],color=clr[band,:3]*.7)
    plt.xlim(xmin+W*xx)
    plt.ylim(F[1],fshow)
    plt.ylabel('Frequency (Hz)]')
    plt.xlabel('Time (sec)')
    plt.title('Spectrogram')
    plt.show()

In [ ]:
w_chan2 = widgets.Select(options=list(zip(channel_site,np.arange(NChannels))),value=0,rows=NChannels)
w_band2 = widgets.Select(options=list(zip(waves,np.arange(nband))),value=0,rows=nband)
w_px2 = widgets.FloatSlider(value=0.5,min=0.5,max=0.5,step=npseg/(N*4)/10,readout=False,continuous_update=False,orientation='horizontal',layout=xslider)
w_py2 = widgets.FloatSlider(value=0.5,min=0.5,max=0.5,step=0.001,readout=False,continuous_update=False,orientation='vertical',layout=yslider)
w_py3 = widgets.FloatSlider(value=0.5,min=0.5,max=0.5,step=0.001,readout=False,continuous_update=False,orientation='vertical',layout=yslider)
w_pw2 = widgets.FloatLogSlider(value=1,base=10,min=0,max=np.log10(N*4/npseg),step=0.02,readout=False,continuous_update=False,orientation='horizontal',layout=xslider)
w_ph2 = widgets.FloatLogSlider(value=1,base=10,min=0,max=2,step=0.02,readout=False,continuous_update=False,orientation='vertical',layout=yslider)
w_ph3 = widgets.FloatLogSlider(value=1,base=10,min=0,max=2,step=0.02,readout=False,continuous_update=False,orientation='vertical',layout=yslider)
w_update2 = widgets.ToggleButton(value=False,description='Continuous update',button_style='info',icon='repeat')

def cont_update2(*args):
    if w_update2.value:
        w_update2.icon = 'times'
        w_px2.continuous_update = True; w_pw2.continuous_update = True
        w_py2.continuous_update = True; w_ph2.continuous_update = True
        w_py3.continuous_update = True; w_ph3.continuous_update = True
    else:
        w_update2.icon = 'repeat'
        w_px2.continuous_update = False; w_pw2.continuous_update = False
        w_py2.continuous_update = False; w_ph2.continuous_update = False
        w_py3.continuous_update = False; w_ph3.continuous_update = False
w_update2.observe(cont_update2,'value')

def update_px2(*arg):
    w_px2.min = max(1/w_pw2.value/2,w_px2.value-1/w_pw2.value)
    w_px2.max = min(1-1/w_pw2.value/2,w_px2.value+1/w_pw2.value)
w_pw2.observe(update_px2,'value')
w_px2.observe(update_px2,'value')

def update_py2(*arg):
    w_py2.min = max(1/w_ph2.value/2,w_py2.value-1/w_ph2.value)
    w_py2.max = min(1-1/w_ph2.value/2,w_py2.value+1/w_ph2.value)
w_ph2.observe(update_py2,'value')
w_py2.observe(update_py2,'value')

def update_py3(*arg):
    w_py3.min = max(1/w_ph3.value/2,w_py3.value-1/w_ph3.value)
    w_py3.max = min(1-1/w_ph3.value/2,w_py3.value+1/w_ph3.value)
w_ph3.observe(update_py3,'value')
w_py3.observe(update_py3,'value')

around = Layout(align_items='flex-start',justify_content='space-around')
out2 = interactive_output(plot_waves,{'channel':w_chan2,'band':w_band2,'px':w_px2,'py1':w_py2,'py2':w_py3,'pw':w_pw2,'ph1':w_ph2,'ph2':w_ph3})
ui2 = VBox([ HBox([ VBox([Label('Select Channel'),w_chan2]),VBox([Label('Select Rhythm'),w_band2]) ]),
           HBox([VBox([Label('Zoom in/out'),Label('X positon')]),VBox([w_pw2,w_px2]),w_update2],layout=around),
           HBox([out2, VBox([HBox([Label('Y positon'),Label('Zoom in/out')]),HBox([w_py2,w_ph2]),HBox([w_py3,w_ph3])]) ],layout=around) ])

display(ui2)